In [ ]:
!pip install conllu
import conllu
import torch
import torch.nn as nn
import json
import math, string, re

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
dev = open("/gdrive/MyDrive/UP_German/de-up-dev.conllu.json",mode="r",encoding="utf-8")
datadev = json.load(dev)
test = open("/gdrive/MyDrive/UP_German/de-up-test.conllu.json",mode="r",encoding="utf-8")
datatest = json.load(test)
train = open("/gdrive/MyDrive/UP_German/de-up-train.conllu.json",mode="r",encoding="utf-8")
datatrain = json.load(train)

In [ ]:
sentence = []
listlemma = []
listupos = []
k = 0
list1 = []
for key, value in datadev.items():
    for i,j in value.items():
      if i == 'conllu':
        for conkey,conval in value.items():
          a = []
          m = ' '
          for m in range(0,len(conval)):
            word = conval[m].split('\t')
            if len(word)>1:
              a.append(word[1])
              listlemma.append([word[1],word[3]])
              listupos.append(word[3])
          if(len(listlemma)>1):
            list1.append(listlemma)
            listlemma = []
          if len(a) > 0:
            m = ' '.join(a)
            sentence.append(m) 


In [ ]:
def features(sent,i):
  word = sent[i][0]
  features = {
        'bias': 1.0,
        'word': word,
        'len(word)': len(word),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit(),
    }
        if i < len(sent)-1:
        wordnext = sent[i+1][0]
        features.update({
            '+1:word': word1,
            '+1:len(word)': len(wordnext),
            '+1:word[:3]': wordnext[:3],
            '+1:word[:2]': wordnext[:2],
            '+1:word[-3:]': wordnext[-3:],
            '+1:word[-2:]': wordnext[-2:],
            '+1:word.ispunctuation': (wordnext in string.punctuation),
            '+1:word.isdigit()': wordnext.isdigit(),
        })
    else:
        features['EOS'] = True

        if i > 0:
        wordprev = sent[i-1][0]
        features.update({
            '-1:word': wordprev,
            '-1:len(word)': len(wordprev),
            '-1:word[:3]': wordprev[:3],
            '-1:word[:2]': wordprev[:2],
            '-1:word[-3:]': wordprev[-3:],
            '-1:word[-2:]': wordprev[-2:],
            '-1:word.ispunctuation': (wordprev in string.punctuation),
            '-1:word.isdigit()': wordprev.isdigit(),

        })
     else:
        features['BOS'] = True


In [ ]:
Xtrain = [features(s) for s in list1]
ytrain = [labels(s) for s in list1]

In [ ]:
def sentencefeatures(sent):
    return [features(sent,i) for i in range(len(sent))]
def wordlabels(sent):
    return [word[1] for word in sent]

In [ ]:
print(list1)

[[['Meine', 'PRON'], ['Mutter', 'NOUN'], ['ist', 'VERB'], ['bettlägerig', 'ADJ'], ['und', 'CONJ'], ['er', 'PRON'], ['kam', 'VERB'], ['einfach', 'ADV'], ['mit', 'ADP'], ['seinen', 'PRON'], ['Geräten', 'NOUN'], ['zu', 'ADP'], ['ihr', 'PRON'], ['nach', 'ADP'], ['Hause', 'NOUN'], ['!', 'PUNCT']], [['Für', 'ADP'], ['uns', 'PRON'], ['war', 'VERB'], ['vor', 'ADP'], ['allem', 'PRON'], ['der', 'DET'], ['Hausbesuch', 'NOUN'], ['das', 'DET'], ['Highlight', 'NOUN'], ['dieses', 'PRON'], ['Optikers', 'NOUN'], ['.', 'PUNCT']], [['Hab', 'AUX'], ["'", 'PUNCT'], ['ich', 'PRON'], ['vorher', 'ADV'], ['noch', 'ADV'], ['von', 'ADP'], ['keinem', 'PRON'], ['Optiker', 'NOUN'], ['geboten', 'VERB'], ['bekommen', 'VERB'], ['.', 'PUNCT']], [['Meine', 'PRON'], ['Mutter', 'NOUN'], ['ist', 'VERB'], ['bettlägerig', 'ADJ'], ['und', 'CONJ'], ['er', 'PRON'], ['kam', 'VERB'], ['einfach', 'ADV'], ['mit', 'ADP'], ['seinen', 'PRON'], ['Geräten', 'NOUN'], ['zu', 'ADP'], ['ihr', 'PRON'], ['nach', 'ADP'], ['Hause', 'NOUN'], ['!